In [24]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# read the Excel file into a pandas dataframe
df = pd.read_excel('12859_2019_2897_MOESM2_ESM.xlsx', sheet_name=None)

# loop through each sheet in the dataframe
for sheet_name, sheet_data in df.items():
    # convert the sheet to a NumPy array
    adj_matrix = sheet_data.to_numpy()
    
    # create a graph from the adjacency matrix
    G = nx.from_numpy_array(adj_matrix)
    
    # draw the graph and save it as an image file
    nx.draw(G, with_labels=True)
    plt.savefig(f"{sheet_name}.png")
    
    # print the nodes and edges of the graph
    print(f"Graph from sheet '{sheet_name}' saved as '{sheet_name}.png'")

NetworkXError: Adjacency matrix not square: nx,ny=(1251, 1252)

In [31]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# read the Excel file into a pandas dataframe
df = pd.read_excel('12859_2019_2897_MOESM2_ESM.xlsx', sheet_name=True)

# loop through each sheet in the dataframe
for sheet_name, sheet_data in df.items():

    # create a graph from the adjacency matrix
    G = nx.from_numpy_array(adj_matrix)
    
    # draw the graph and save it as an image file
    nx.draw(G, with_labels=True)
    plt.savefig(f"{sheet_name}.png")
    
    # print the nodes and edges of the graph
    print(f"Graph from sheet '{sheet_name}' saved as '{sheet_name}.png'")

NetworkXError: Adjacency matrix not square: nx,ny=(880, 881)

In [29]:
df['E.coli (iJ01366)'].shape()


TypeError: 'tuple' object is not callable